In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/abalone.csv")

df.head()

,id,Sex,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight,Rings
0,0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


In [6]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_absolute_error
from tensorflow.keras import layers, models

In [7]:
# 'Sex' 열을 라벨 인코딩하여 숫자로 변환
label_encoder = LabelEncoder()
df['Sex'] = label_encoder.fit_transform(df['Sex'])  # 성별 정보를 숫자형으로 변환하여 모델이 처리

In [8]:
# 입력 변수(X)와 출력 변수(y)를 분리
X = df.drop('Rings', axis=1)   # 'Rings' 열은 예측하고자 하는 값이므로 독립 변수에서 제거
y = df['Rings']   # 'Rings' 열을 종속 변수로 설정

In [9]:
# 입력 변수(X)를 표준화
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)   # 모든 입력 변수를 표준화하여 각 변수의 평균을 0, 분산을 1

In [10]:
# CNN 모델에 입력하기 위해 입력 데이터를 4차원으로 변환
num_features = X_scaled.shape[1]
X_scaled = X_scaled.reshape(X_scaled.shape[0], num_features, 1, 1)

In [11]:
# 학습 데이터와 테스트 데이터로 분할
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)   # CNN 모델에 맞도록 입력 데이터를 4차원 형태로 변환

In [12]:
print(X_train.shape, X_test.shape)

(3341, 9, 1, 1) (836, 9, 1, 1)


In [13]:
# CNN 모델
model = models.Sequential([
     # 첫 번째 컨볼루션 레이어: 32개의 필터를 사용하고 ReLU 활성화 함수를 사용
    layers.Conv2D(32, (1, 1), activation='relu', input_shape=(3, 3, 1), padding='same'),
     # 최대 풀링 레이어: 특성 맵의 크기를 줄여 연산량을 줄임
    layers.MaxPooling2D((1, 1)),
     # 두 번째 컨볼루션 레이어: 64개의 필터를 사용
    layers.Conv2D(64, (1, 1), activation='relu', padding='same'),
     # 다차원 배열을 1차원으로 변환하여 Dense 레이어에 전달
    layers.Flatten(),
     # 완전 연결층: 64개의 뉴런과 ReLU 활성화 함수를 사용
    layers.Dense(64, activation='relu'),
      # 출력층: 예측하려는 'Rings' 값이 연속형이므로 선형 활성화 함수를 사용
    layers.Dense(1, activation='linear')
])

# 모델을 컴파일
# Adam 옵티마이저를 사용하고, 손실 함수로 평균 제곱 오차(MSE)를, 평가 지표로 평균 절대 오차(MAE)를 사용
model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['mae'])

# 학습 데이터로 20 에포크 동안 모델을 학습하고, 검증 데이터로 성능을 평가
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test))


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 51.9808 - mae: 5.8224 - val_loss: 6.5264 - val_mae: 1.9145
Epoch 2/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 5.8050 - mae: 1.7365 - val_loss: 5.5887 - val_mae: 1.7231
Epoch 3/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5.7820 - mae: 1.7425 - val_loss: 5.1538 - val_mae: 1.6501
Epoch 4/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 5.2038 - mae: 1.6363 - val_loss: 4.9594 - val_mae: 1.5873
Epoch 5/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 4.6716 - mae: 1.5386 - val_loss: 4.8824 - val_mae: 1.6161
Epoch 6/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 4.8553 - mae: 1.5576 - val_loss: 4.7278 - val_mae: 1.5792
Epoch 7/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4.7323 - mae: 1.5721 - val_loss: 5.0676 - val_mae: 1.7466
Epoch 8/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4.3343 - mae: 1.4982 - val_loss: 4.7216 - val_mae: 1.6098
Epoch 9/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - l

In [14]:
# 테스트 데이터에 대한 예측을 수행
y_pred = model.predict(X_test)  # 테스트 데이터를 사용하여 모델이 예측한 'Rings' 값을 출력

27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


In [15]:
# 모델의 평균 절대 오차(MAE)를 계산하여 출력
mae = mean_absolute_error(y_test, y_pred)   # 실제 값과 예측 값의 평균 절대 오차를 계산하여 모델의 성능을 평가
print(f"\n모델의 평균 절대 오차 (MAE): {mae}")


모델의 평균 절대 오차 (MAE): 1.6495490624573812


In [16]:
model.save('/content/drive/MyDrive/cnn_abalone_model.h5')

In [17]:
from tensorflow.keras.models import load_model

pretrained_model = load_model('/content/drive/MyDrive/cnn_abalone_model.h5')

In [18]:
# 미세 조정을 위해 마지막 두 개의 레이어를 제외한 모든 레이어의 학습을 동결
for layer in pretrained_model.layers[:-2]:
    layer.trainable = False  # 학습을 동결하여 이전에 학습된 가중치를 유지하고, 새로운 레이어만 학습하도록 설정

In [19]:
from tensorflow.keras import layers, models

# 새로운 모델을 정의하고, 불러온 모델의 레이어를 추가
new_model = models.Sequential(pretrained_model.layers)  # 이전 모델의 레이어를 새로운 모델에 추가
new_model.add(layers.Dense(128, activation='relu'))  # 새로운 완전 연결층을 추가하여 모델의 학습 능력을 확장
new_model.add(layers.Dense(1, activation='linear'))  # 최종 출력층을 추가하여 'Rings' 값을 예측


new_model.compile(optimizer='adam',
                  loss='mean_squared_error',
                  metrics=['mae'])

# 미세 조정(fine-tuning)을 통해 추가된 레이어를 학습
history_finetune = new_model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test))


Epoch 1/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 46.9715 - mae: 5.5288 - val_loss: 5.1095 - val_mae: 1.6692
Epoch 2/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 4.5318 - mae: 1.5444 - val_loss: 4.8243 - val_mae: 1.5243
Epoch 3/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4.3643 - mae: 1.5171 - val_loss: 4.7204 - val_mae: 1.6298
Epoch 4/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 4.5443 - mae: 1.5306 - val_loss: 4.6541 - val_mae: 1.6076
Epoch 5/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 4.4438 - mae: 1.5178 - val_loss: 4.5039 - val_mae: 1.5385
Epoch 6/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 4.0354 - mae: 1.4444 - val_loss: 4.5505 - val_mae: 1.5906
Epoch 7/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4.2607 - mae: 1.4859 - val_loss: 4.4611 - val_mae: 1.4982
Epoch 8/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 4.1883 - mae: 1.4656 - val_loss: 4.4392 - val_mae: 1.5289
Epoch 9/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - l

In [20]:
y_pred = new_model.predict(X_test)

27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


In [21]:
mae = mean_absolute_error(y_test, y_pred)
print(f"\n모델의 평균 절대 오차 (MAE): {mae}")


모델의 평균 절대 오차 (MAE): 1.4833381692188208
